# Lecture 8: Ligand-Receptor Interactions - SOLUTION

**Date:** December 27, 2025

---

In [ ]:
import scanpy as sc
import liana as li
import numpy as np

# Load data with annotations
adata = sc.datasets.pbmc3k_processed()
print(f"Loaded: {adata.n_obs} cells × {adata.n_vars} genes")

## Task 1: Cell-Cell Communication (35 points)

In [ ]:
# Run LIANA
li.mt.rank_aggregate(adata, groupby='louvain', use_raw=False, verbose=True)

# Filter significant interactions
liana_res = adata.uns['liana_res']
significant = liana_res[liana_res['magnitude_rank'] < 0.01]
print(f"Significant interactions: {len(significant)}")

# Visualize heatmap
li.pl.dotplot(adata=adata, colour='magnitude_rank', size='specificity_rank',
              source_labels=['CD4 T', 'CD14 Monocytes'], 
              target_labels=['CD4 T', 'CD14 Monocytes', 'B'])

print("Communication analysis complete!")

## Task 2: Ligand-Receptor Networks (30 points)

In [ ]:
# Top interactions
top_lr = significant.nsmallest(20, 'magnitude_rank')
print("Top 20 ligand-receptor pairs:")
print(top_lr[['source', 'target', 'ligand_complex', 'receptor_complex', 'magnitude_rank']])

# Focus on T cell-monocyte
tcell_mono = significant[
    (significant['source'].str.contains('T')) & 
    (significant['target'].str.contains('Monocyte'))
]

print(f"\nT cell-Monocyte interactions: {len(tcell_mono)}")

# Dot plot
li.pl.dotplot(adata=adata, colour='magnitude_rank',
              source_labels=['CD4 T', 'CD8 T'],
              target_labels=['CD14 Monocytes', 'FCGR3A Monocytes'])

print("Network analysis complete!")

## Task 3: Differential Abundance (35 points)

In [ ]:
from scipy.stats import chi2_contingency
import pandas as pd

# Simulate two conditions
adata.obs['condition'] = np.random.choice(['Control', 'Treatment'], adata.n_obs)

# Calculate proportions
ct = pd.crosstab(adata.obs['louvain'], adata.obs['condition'])
print("Cell type counts per condition:")
print(ct)

# Chi-square test
chi2, pval, dof, expected = chi2_contingency(ct)
print(f"\nChi-square test: p={pval:.4f}")

# Visualize composition
ct_norm = ct.div(ct.sum(axis=0), axis=1)
ct_norm.plot(kind='bar', stacked=True, figsize=(10, 6))

print("Differential abundance analysis complete!")

---

## Summary

- ✓ Cell-cell communication with LIANA
- ✓ Ligand-receptor pair identification
- ✓ Differential abundance testing